In [1]:
#!pip install kaggle

In [2]:
#importing kaggle and getting the dataset from kaggle api
import kaggle 
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
#importing zipfile to extract the dataset 
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
# importing pandas
import pandas as pd

In [5]:
# reading the csv and handling the null values 
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [6]:
df.head(3)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5


In [7]:
#Renaming the columns
#method 1 df.rename(columns = {'Order Id':'order_id','City':'city'})

#method 2
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(' ','_')

In [8]:
df.head(2)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3


In [9]:
#derive new column: discount, sale price, and profit
df['discount'] = df['list_price']*(df['discount_percent']/100)
df['sale_price'] = df['list_price']-df['discount']
df['profit'] = df['sale_price']-df['cost_price']

In [10]:
df.head(3)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5


In [11]:
#convert order date from object data type to datatime
df.dtypes
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d')

In [12]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [13]:
#dropping cost price, list price, discount percent columns
df.drop(['cost_price','list_price','discount_percent'], axis = 1, inplace = True)

In [14]:
#pip install sqlalchemy pyodbc

In [18]:
#load the data into sql server using replace option
import sqlalchemy as sal
#engine = sal.create_engine('msql://Aditya/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
engine = sal.create_engine('mssql+pyodbc://Aditya/master?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

conn=engine.connect()

In [20]:
#load the data into sql server using append option
df.to_sql('df_orders',con=conn, index=False, if_exists = 'append')

38